In [1]:
from pathlib import Path

import h5py
import numpy as np
import open3d as o3d
import pandas as pd
import pycolmap
from deep_image_matching.thirdparty.transformations import (
    affine_matrix_from_points,
    decompose_matrix,
)
from deep_image_matching.triangulation import db_from_existing_poses
from deep_image_matching.utils import COLMAPDatabase, OutputCapture

root_path = Path("datasets/belv_20230725")
image_dir = root_path / "images"

sfm_dir = root_path / "results_superpoint+lightglue_bruteforce_quality_highest"
sfm_rec_path = sfm_dir / "reconstruction"

# Output path for the triangulated markers
output_path = root_path / "marker_triang"
db_path = output_path / "database_markers.db"

# Image coordinates of the markers
markers_file = root_path / "markers_image_20230725.csv"

# World coordinates of the markers
georef_points = root_path / "markers_utm.csv"

output_path.mkdir(exist_ok=True, parents=True)

# Get image list
images = sorted(image_dir.glob("*"))

# Define a subset of the markers to use (leve none to use all)
# markers_to_use = ["D38", "T2", "F2", "F4", "F10", "F20"]
markers_to_use = ["D38", "F2", "F4", "F10", "F20"]


# Dense reconstruction path to georeference
dense_rec_path = root_path / "results_roma_bruteforce_quality_high/dense_model"

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Deep Image Matching loaded in 2.726 seconds.


In [2]:
# Read marker image coordinates and create keypoints dictionary with the form:
# {"image_name": keypoints_array}
# {
#     "image1.jpg": np.array([[x1, y1], [x2, y2], ...]),
#     "image2.jpg": np.array([[x1, y1], [x2, y2], ...]),
#     ...
# }
markers_image = pd.read_csv(markers_file, header=None, names=["image", "marker", "x", "y"])
markers_image.sort_values(["image", "marker"], inplace=True, ascending=True)
if markers_to_use:
    markers_image = markers_image[markers_image["marker"].isin(markers_to_use)]

kpts = {}
for image, gr in markers_image.groupby("image"):
    image = image + ".JPG"
    kpts[image] = gr[["x", "y"]].values
for k, v in kpts.items():
    print(k, ":\n", v)

# Manually create 1-to-1 matches array
ids = np.arange(0, len(kpts[images[0].name]))
matches_idx = np.array([ids, ids]).astype(np.int64).T
print("matches idx:\n", matches_idx)

p1_20230725_115953_IMG_1147.JPG :
 [[4832.3403 1582.9143]
 [2027.8405 3478.0388]
 [ 725.7628 3845.6572]
 [4092.3181 2256.6533]
 [3416.7617 3300.4646]]
p2_20230725_120026_IMG_0885.JPG :
 [[3194.6826 1729.3799]
 [3573.5344 3204.1846]
 [4105.1738 3556.3171]
 [4139.4536 1980.674 ]
 [5928.5713 2516.8994]]
matches idx:
 [[0 0]
 [1 1]
 [2 2]
 [3 3]
 [4 4]]


In [3]:
# # plot image with markers
# import cv2
# from matplotlib import pyplot as plt

# image_id = 1

# img = cv2.cvtColor(cv2.imread(str(images[image_id])), cv2.COLOR_BGR2RGB)
# plt.imshow(img)
# plt.scatter(
#     kpts[images[image_id].name][:, 0],
#     kpts[images[image_id].name][:, 1],
#     c="r",
#     s=10,
# )

In [4]:
# Save features to h5 file
features_h5 = output_path / "features.h5"
with h5py.File(features_h5, "w") as f:
    for image in images:
        image_name = image.name
        kp = kpts[image_name]
        f.create_group(image_name)
        f[image_name].create_dataset("keypoints", data=kp, dtype=np.float32)

# Save matches to h5 file
matches_h5 = output_path / "matches.h5"
with h5py.File(matches_h5, "w") as f:
    image0, image1 = images[0].name, images[1].name
    gr0 = f.create_group(image0)
    gr0.create_dataset(image1, data=matches_idx, dtype=np.int64)

pair_file = sfm_dir / "pairs.txt"

# # print features_h5 content
# with h5py.File(features_h5, "r") as f:
#     print(f.keys())
#     print(f[images[0].name].keys())
#     print(f[images[0].name]["keypoints"][:])
#     print(f[images[1].name].keys())
#     print(f[images[1].name]["keypoints"][:])

# # Print matches.h5 content
# with h5py.File(matches_h5, "r") as f:
#     print(f.keys())
#     g0 = f[images[0].name]
#     print(g0.keys())
#     g1 = g0[images[1].name]
#     print(g1.__array__())

In [5]:
sfm_rec = pycolmap.Reconstruction(sfm_rec_path)

# Create a new database with the dense features and the known camera poses

db_from_existing_poses(
    db_path,
    features_h5,
    matches_h5,
    sfm_rec_path,
    pair_file,
    do_geometric_verification=False,
)

2024-04-12 12:06:35 | [WARNING ] The database already exists, deleting it.


Importing matches: 100%|██████████| 1/1 [00:00<00:00, 1454.34it/s]


In [6]:
# Define the options for the triangulation according to the IncrementalPipelineOptions available in pycolmap
# print(pycolmap.IncrementalPipelineOptions().summary())
opt = dict(
    min_num_matches=3,
    triangulation=dict(
        ignore_two_view_tracks=False,
    ),
)

In [7]:
# Run the triangulation with the known camera poses
verbose = True
with OutputCapture(verbose):
    with pycolmap.ostream():
        reconstruction = pycolmap.triangulate_points(
            sfm_rec,
            db_path,
            image_dir,
            output_path,
            options=opt,
        )

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  2.837421e+00    0.00e+00    1.32e+03   0.00e+00   0.00e+00  1.00e+04        0    3.00e-05    9.20e-05
   1  2.471128e+00    3.66e-01    1.42e-01   5.44e-04   1.00e+00  3.00e+04        0    4.39e-05    4.82e-04


I20240412 12:06:35.960026 3086923 misc.cc:198] 
Loading database
I20240412 12:06:35.961102 3086923 database_cache.cc:54] Loading cameras...
I20240412 12:06:35.961148 3086923 database_cache.cc:64]  2 in 0.000s
I20240412 12:06:35.961160 3086923 database_cache.cc:72] Loading matches...
I20240412 12:06:35.961179 3086923 database_cache.cc:78]  1 in 0.000s
I20240412 12:06:35.961184 3086923 database_cache.cc:94] Loading images...
I20240412 12:06:35.961223 3086923 database_cache.cc:143]  2 in 0.000s (connected 2)
I20240412 12:06:35.961228 3086923 database_cache.cc:154] Building correspondence graph...
I20240412 12:06:35.961241 3086923 database_cache.cc:190]  in 0.000s (ignored 0)
I20240412 12:06:35.961364 3086923 timer.cc:91] Elapsed time: 0.000 [minutes]
I20240412 12:06:35.961529 3086923 misc.cc:198] 
Triangulating image #1 (0)
I20240412 12:06:35.961535 3086923 sfm.cc:473] => Image sees 0 / 5 points
I20240412 12:06:35.961606 3086923 sfm.cc:478] => Triangulated 5 points
I20240412 12:06:35.9616

In [8]:
pts = reconstruction.points3D
for i, pt in sorted(pts.items()):
    print(i, pt)

1490 Point3D(xyz=[6.80372, -1.44258, 14.1866], color=[143, 139, 86], error=0.0227758, track=Track(length=2))
1491 Point3D(xyz=[1.99842, 1.31624, 8.79684], color=[163, 107, 101], error=0.358146, track=Track(length=2))
1492 Point3D(xyz=[0.691425, 1.45129, 6.99773], color=[244, 228, 229], error=1.49442, track=Track(length=2))
1493 Point3D(xyz=[4.91828, -0.280677, 10.1311], color=[234, 242, 242], error=0.00433232, track=Track(length=2))
1494 Point3D(xyz=[3.78636, 0.790003, 6.84661], color=[254, 243, 251], error=0.143108, track=Track(length=2))


In [9]:
img = reconstruction.images[1]
img.points2D

[Point2D(xy=[4832.84, 1583.41], point3D_id=1490),
 Point2D(xy=[2028.34, 3478.54], point3D_id=1491),
 Point2D(xy=[726.263, 3846.16], point3D_id=1492),
 Point2D(xy=[4092.82, 2257.15], point3D_id=1493),
 Point2D(xy=[3417.26, 3300.96], point3D_id=1494)]

In [10]:
image_id = 1
local = []
for pt in reconstruction.images[image_id].points2D:
    if not isinstance(pt.point3D_id, int) or pt.point3D_id < 0:
        print(f"Point {pt.point2D_idx} is not triangulated")
        continue
    local.append(reconstruction.points3D[pt.point3D_id].xyz)
local = np.array(local)
print(local)

[[ 6.80371791 -1.44258202 14.18655572]
 [ 1.99842367  1.31624029  8.79683948]
 [ 0.69142514  1.45128844  6.99772945]
 [ 4.91827547 -0.28067699 10.13106025]
 [ 3.78635656  0.79000304  6.8466085 ]]


In [11]:
# Read the georeferenced points
georef = pd.read_csv(georef_points)
georef.index = georef["Label"]
georef.drop(columns=["Label"], inplace=True)
georef.sort_values("Label", inplace=True)
if markers_to_use:
    georef = georef[georef.index.isin(markers_to_use)]
print(georef)

                   X             Y            Z
Label                                          
D38    416273.191792  5.091048e+06  1912.062781
F10    416458.548600  5.091086e+06  1841.996800
F2     416514.111700  5.091104e+06  1839.217200
F20    416376.149300  5.091104e+06  1883.523600
F4     416451.502700  5.091155e+06  1857.065600


In [12]:
# Estimate a Helmert transformation between the mean pcd and the ref.
T = affine_matrix_from_points(
    local.T, georef.to_numpy().T, shear=False, scale=True, usesvd=True
)
scale, _, angles, translation, _ = decompose_matrix(T)
scale_percent = scale.mean() - 1
angles_deg = np.rad2deg(angles)

print(f"Translation: {translation} m")
print(f"Angles: {angles_deg} deg")
print(f"Scale: {scale_percent:.6}%")

Translation: [4.16637855e+05 5.09123835e+06 1.88055817e+03] m
Angles: [-91.10481171  -0.31795999 143.15041448] deg
Scale: 25.1244%


In [13]:
# Export the dense point cloud from roma to a ply file
dense_rec = pycolmap.Reconstruction(dense_rec_path)
dense_rec.export_PLY(dense_rec_path / "dense.ply")

In [14]:
# Read and transform the point cloud
pcd = o3d.io.read_point_cloud(str(dense_rec_path / "dense_merged_clean.ply"))
georef_pcd = pcd.transform(T)
o3d.io.write_point_cloud(str(dense_rec_path / "dense_georef.ply"), georef_pcd)

True

In [15]:
dense_rec_path

PosixPath('datasets/belv_20230725/results_roma_bruteforce_quality_high/dense_model')